In [1]:
import json

file_path = "4-baseline.json"

with open(file_path, 'r') as file:
    data = json.load(file)

print(len(data))
print(data[0])

31000
[{'frame_idx': 0, 'track_id': 0, 'left': 804, 'top': 307, 'right': 893, 'bottom': 396}, {'frame_idx': 0, 'track_id': 1, 'left': 447, 'top': 304, 'right': 519, 'bottom': 353}, {'frame_idx': 0, 'track_id': 2, 'left': 1582, 'top': 347, 'right': 1620, 'bottom': 373}, {'frame_idx': 0, 'track_id': 3, 'left': 1654, 'top': 423, 'right': 1701, 'bottom': 453}, {'frame_idx': 0, 'track_id': 4, 'left': 1538, 'top': 362, 'right': 1579, 'bottom': 392}, {'frame_idx': 0, 'track_id': 5, 'left': 479, 'top': 358, 'right': 539, 'bottom': 450}, {'frame_idx': 0, 'track_id': 6, 'left': 917, 'top': 356, 'right': 987, 'bottom': 393}, {'frame_idx': 0, 'track_id': 7, 'left': 605, 'top': 280, 'right': 684, 'bottom': 332}, {'frame_idx': 0, 'track_id': 8, 'left': 514, 'top': 333, 'right': 586, 'bottom': 408}, {'frame_idx': 0, 'track_id': 9, 'left': 1624, 'top': 351, 'right': 1661, 'bottom': 380}, {'frame_idx': 0, 'track_id': 10, 'left': 767, 'top': 304, 'right': 802, 'bottom': 337}, {'frame_idx': 0, 'track_id'

In [2]:
import numpy as np
from math import isnan

def detect_turn_left(track_coordinates):
    if len(track_coordinates) < 3:
        return "Not enough data to determine"

    angles = []
    for i in range(1, len(track_coordinates) - 1):
        vector1 = track_coordinates[i - 1] - track_coordinates[i]
        vector2 = track_coordinates[i + 1] - track_coordinates[i]

        # Calculate the angle between the vectors
        angle = np.arccos(np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2)))

        if isnan(angle) == False:
            # Convert angle to degrees
            angle_deg = np.degrees(angle)
            if isnan(angle_deg) == False:
                angles.append(angle_deg)

    # Calculate the average angle
    avg_angle = np.mean(angles)


    # Determine if the car is turning or going straight
    if avg_angle >= 45 and avg_angle <= 135:
        return True
    else:
        return False


In [3]:
def process(data, frame, track_num):
    res = []
    while frame < len(data):
        frame_info = data[frame]
        flag = False
        for track_info in frame_info:
            if track_info['track_id'] == track_num:
                flag = True
                break
        if flag == False:
            break
        centroid = [0,0]
        centroid[0] = (track_info['left'] + track_info['right'])/2
        centroid[1] = (track_info['top'] + track_info['bottom'])/2
        res.append(centroid)
        frame += 1
    return np.array(res)
        

In [4]:
def event_detect(data, start_frame, track_num, event = "turn-left"):
    # get the numpy array of all the coordinate sequences
    sequence = process(data, start_frame, track_num)

    # run left detection
    res = False
    if event == "turn-left":
        res = detect_turn_left(sequence)
    return res

In [5]:
from tqdm import tqdm
def run_check(file_path):
    seen_track = []
    event_occurance = 0
    with open(file_path, 'r') as file:
        data = json.load(file)
    for i in tqdm(range(len(data))):
        frame = data[i]
        if frame == None:
            continue
        for track in frame:
            if track['track_id'] not in seen_track:
                if event_detect(data, i, track['track_id']):
                    event_occurance += 1
                seen_track.append(track['track_id'])
    return event_occurance, len(data)
    

In [6]:
for i in range(6):
    file_path = str(i)+"-baseline.json"
    occurance, frame_num = run_check(file_path)
    print("Running File ",i, "has average event occurance per frame of ", occurance/frame_num)

  0%|                                                 | 0/31000 [00:00<?, ?it/s]/var/folders/39/fvsq4djn0lz6rfqd7738typc0000gq/T/ipykernel_39575/3113355297.py:14: RuntimeWarning: invalid value encountered in scalar divide
  angle = np.arccos(np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2)))
/var/folders/39/fvsq4djn0lz6rfqd7738typc0000gq/T/ipykernel_39575/3113355297.py:14: RuntimeWarning: invalid value encountered in arccos
  angle = np.arccos(np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2)))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████████████████████████████

Running File  0 has average event occurance per frame of  0.05835483870967742


100%|███████████████████████████████████| 31000/31000 [00:03<00:00, 9112.75it/s]


Running File  1 has average event occurance per frame of  0.05116129032258065


100%|███████████████████████████████████| 31000/31000 [00:04<00:00, 6233.83it/s]


...

Running File  3 has average event occurance per frame of  0.08038709677419355


100%|███████████████████████████████████| 31000/31000 [00:07<00:00, 4309.50it/s]


Running File  4 has average event occurance per frame of  0.10619354838709677


100%|███████████████████████████████████| 25052/25052 [00:05<00:00, 4678.08it/s]

Running File  5 has average event occurance per frame of  0.11164777263292353
